# Extreme Volatility

Extreme volatility considers extreme prices during a trading day to descrrbe how volatile a stock is. In this notebook, four kinds of extreme volatilities are introduced. The data used here is minute data with high, low, open and close.     
The first three are Parkinson(1980), Garman Klass (1980) and Rogers and satchell (1991) intraday volatility. All of them have solid theory to back up. They all focus on information of every minute's high, low, open and close price. There formulas are as below:  
Parkinson (1980):  
$   \begin{equation*} \sigma = \sqrt{\frac{1}{4Nln2}\sum_{i = 1}^N(ln\frac{h_i}{l_i})^2} \end{equation*}$  
Garman Klass (1980):  
$ \begin{equation*} \sigma = \sqrt{\frac{1}{N}\sum_{i = 1}^N\frac{1}{2}(ln\frac{h_i}{l_i})^2 - \frac{1}{N}\sum_{i = 1}^N(2ln2-1)(ln\frac{c_i}{c_{i-1}})^2}  \end{equation*}$  
Rogers and satchell (1991):  
$   \begin{equation*}  \sigma = \sqrt{\frac{1}{N}\sum_{i = 1}^N(ln\frac{h_i}{l_i})(ln\frac{h_i}{o_i}) + (ln\frac{l_i}{c_{i}}) (ln\frac{l_i}{o_{i}})}  \end{equation*}$  
In all formulas $h_i$, $l_i$, $cl_i$,$o_i$ are high, low, close and open prices for interval i respectively.

 The last intraday volatility is self-defined, which employs trading experience of my own. When observating intraday performance of a stock, people are likely to notice several local high and local low points. Between two continuous local extreme points, there are local trends, which describe how volatile a stock is in short term. For stocks with the same highest price and lowest price of a day, the more short-term trends a stock has, the more severe every short-term trend is, the more volatile a stock should be in a day. According to this observation, I constructed the last metric, which is also similar to metrics described above:  
$   \begin{equation*} \sigma = \sqrt{\sum_{j = 1}^N(ln\frac{extreme_j}{extreme_{j-1}})^2} \end{equation*}$    
where $extreme_j$ denotes the local extreme values of a stock close price trend during a day.



In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as plt
import os
import time

In [5]:
#a class defined to find peak points(local extreme points) 
class peak(object):
#==============================================================================
#find peak points of time series
#peak points decided when two ma lines meet
#MA_length1, MA_length2: window length of two ma   lines
#last: last price of 1 period
#==============================================================================

    def __init__(self,MA_length1,MA_length2):
        self.last=np.array([])
        self.n1=MA_length1
        self.n2=MA_length2
        self.n3=max(self.n1,self.n2)
        self.MA_1=[]
        self.MA_2=[]
        self.section_dict=[(0,'None')]
        self.diff=[0]*(self.n3-1)
        #index of effective peak points
        self.min_peak=[]
        self.max_peak=[]
        #index of all peak points
        self.min_peak_all=[]
        self.max_peak_all=[]
        
        self.min_now=[]
        self.max_now=[]
        self.latest_peak = "None"
        self.trend_inc=[]
        self.trend_dec=[]

    def find_peak(self,data):
        temp=data
        self.last=np.append(self.last,temp)
        self.current_wave=(self.last.max()/self.last.min()-1)
        
        if len(self.last)>=max(self.n1,self.n2):
            self.MA_1.append(self.last[-self.n1:].mean())
            self.MA_2.append(self.last[-self.n2:].mean())
            self.diff.append(self.MA_1[-1]-self.MA_2[-1])
        else:
            self.MA_1.append(self.last[-1])
            self.MA_2.append(self.last[-1])
           
        if len(self.last) <= self.n3:
            self.min_now.append(np.argmin(self.last.min()))
            self.max_now.append(np.argmax(self.last.max()))
            self.trend_inc.append(0)
            self.trend_dec.append(0)

        if len(self.last) > self.n3:
            if self.diff[-1]==0:
                self.diff[-1]=self.diff[-2]
            
            if self.diff[-1]>0 and self.diff[-1]*self.diff[-2]<0 and self.latest_peak != "min":
                self.section_dict.append((len(self.diff)-1,'min'))
                start=self.section_dict[-2][0]
                end=self.section_dict[-1][0]
                self.min_peak_all.append(start+np.argmin(self.last[start:end]))                   
                last_three = [self.last[self.max_now[-1]],self.last[start:end].min(),self.last[self.min_now[-1]]]
                if max(last_three)/min(last_three)<1+self.current_wave/10:
                    if len(self.max_peak)>0: del self.max_peak[-1]
                    if self.last[self.min_now[-1]]>self.last[start:end].min() and len(self.min_peak)>0:
                        self.min_peak[-1] = start+np.argmin(self.last[start:end])
                else:
                    self.min_peak.append(start+np.argmin(self.last[start:end]))
                self.latest_peak = "min"                       
                if len(self.min_peak)>0: self.min_now.append(self.min_peak[-1])
                else: self.min_now.append(self.min_now[-1])
                if len(self.max_peak)>0: self.max_now.append(self.max_peak[-1])
                else: self.max_now.append(self.max_now[-1])

                
            elif self.diff[-1]<0 and self.diff[-1]*self.diff[-2]<0 and self.latest_peak != "max" :
                self.section_dict.append((len(self.diff)-1,'max'))
                start=self.section_dict[-2][0]
                end=self.section_dict[-1][0]
                self.max_peak_all.append(start+np.argmax(self.last[start:end]))
                last_three = [self.last[self.max_now[-1]],self.last[start:end].max(),self.last[self.min_now[-1]]]
                if max(last_three)/min(last_three)<1.0016:     
                    if len(self.min_peak)>0: del self.min_peak[-1]
                    if self.last[self.max_now[-1]]<self.last[start:end].max() and len(self.max_peak)>0:
                        self.max_peak[-1] = start+np.argmax(self.last[start:end])
                else:
                    self.max_peak.append(start+np.argmax(self.last[start:end]))
                self.latest_peak = "max"                       
                if len(self.min_peak)>0: self.min_now.append(self.min_peak[-1])
                else: self.min_now.append(self.min_now[-1])
                if len(self.max_peak)>0: self.max_now.append(self.max_peak[-1])
                else: self.max_now.append(self.max_now[-1])
                
            else:
                self.min_now.append(self.min_now[-1])
                self.max_now.append(self.max_now[-1])

            if len(self.min_peak)>2:             
#                self.trend_inc.append((self.last[self.min_peak[-1]]-self.last[self.min_peak[-2]])/(self.last[self.min_peak[-2]])/((self.min_peak[-1]-self.min_peak[-2])))
                self.trend_inc.append((self.last[self.min_peak[-1]]-self.last[self.min_peak[-2]])/(self.last[self.min_peak[-2]]))                
            else: self.trend_inc.append(0)
            if len(self.max_peak)>2:
#                self.trend_dec.append((self.last[self.max_peak[-1]]-self.last[self.max_peak[-2]])/(self.last[self.max_peak[-2]])/((self.max_peak[-1]-self.max_peak[-2])))
                self.trend_dec.append((self.last[self.max_peak[-1]]-self.last[self.max_peak[-2]])/(self.last[self.max_peak[-2]]))
            else:self.trend_dec.append(0)

In [6]:
def find_peak(stock_data,ma_1=5,ma_2=10):
    peak_data=peak(ma_1,ma_2)
    for index in range(len(stock_data)):
        minute_data=pd.DataFrame(stock_data.iloc[index]).T
        peak_data.find_peak(minute_data.close)
    return peak_data

First, let's see the effect of the local extreme points finding code.

In [8]:
path=os.path.join((os.getcwd())[:-9],'data')
trade_list=os.listdir(path)

#choose 1 dataset as an example
object_stock=trade_list[10]
temp_data=pd.read_csv(os.path.join(path,object_stock))
temp_data.timestamp=temp_data.timestamp.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(x)))
temp_data.set_index('timestamp',inplace=True)
temp_data=temp_data[temp_data.volume!=0]
temp_data.dropna(how='any',inplace=True)
peak_data=find_peak(temp_data)
data=peak_data.last
arr_section=np.array(peak_data.section_dict[:])[:,0].astype('int')

last_data=go.Scatter(x=list(range(len(data))),y=data,mode='lines',name='last_price')
#build up compare data
max_point=data.argmax()
min_point=data.argmin()
compare_index=[0,min(max_point,min_point),max(max_point,min_point),len(data)-1]
compare_data=data[compare_index]
compare_data=go.Scatter(x=compare_index,y=compare_data,mode='lines',name='simulate_price')
peak_min_data=go.Scatter(x=peak_data.min_peak_all,y=data[peak_data.min_peak_all],mode='markers',name='peak_min',marker=dict(size=20,color='green'))
peak_max_data=go.Scatter(x=peak_data.max_peak_all,y=data[peak_data.max_peak_all],mode='markers',name='peak_max',marker=dict(size=20,color='red'))
#section=go.Scatter(x=arr_section,y=data[arr_section],mode='markers',name='section',marker=dict(size=20,color='black',symbol='cross'))
layout=go.Layout(title=object_stock[:-4])
#data_plot=[last_data,peak_min_data,peak_max_data,section]
data_plot=[last_data,peak_min_data,peak_max_data,compare_data]
fig=go.Figure(data=data_plot,layout=layout)
plt.plot(fig)


'temp-plot.html'

In the new window, we can see the minute close price trend of CAT on 23rd, May, 2019. Red points denote local high point the code finds and green points denote local low points. Compared to the simulated price denotes in red line, even though the highest and lowest price of the day are the same, real price is more volatile because of the exsitence of short-term trends, located between two continuous extreme points.  
The following part is to calculate four metrics of extreme volatility.

In [9]:
extreme_vol=pd.DataFrame(columns=['Parkinson','Garman_Klass','Rogers_Satchell','local_peak_vol'])
for object_stock in trade_list:
    temp_data=pd.read_csv(os.path.join(path,object_stock))
    temp_data.timestamp=temp_data.timestamp.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(x)))
    temp_data.set_index('timestamp',inplace=True)
    temp_data=temp_data[temp_data.volume!=0]
    temp_data.dropna(how='any',inplace=True)

    #parkinson ,Garman_Klass, Rogers_Satchell extreme vol
    part1=(np.log(temp_data.high/temp_data.low)**2).sum()
    extreme_vol.loc[object_stock[:-7],'Parkinson']=(part1/(4*len(temp_data.high)*np.log(2)))**0.5

    part1=(np.log((temp_data.high/temp_data.low).iloc[1:])**2).sum()
    part2=(2*np.log(2)-1)*(np.log((temp_data.close/temp_data.close.shift()).iloc[1:])**2).sum()
    extreme_vol.loc[object_stock[:-7],'Garman_Klass']=((part1/2-part2)/len(temp_data.high))**0.5

    extreme_vol.loc[object_stock[:-7],'Rogers_Satchell']=(((np.log(temp_data.high/temp_data.low)*np.log(temp_data.high/temp_data.open)+np.log(temp_data.low/temp_data.close)*np.log(temp_data.low/temp_data.open)).sum())/len(temp_data.high))**0.5

    #find local extreme data
    peak_data=find_peak(temp_data)
    #combine all the local extreme index
    index_all=[0]+peak_data.min_peak+peak_data.max_peak+[len(peak_data.last)-1]
    index_all.sort()
    extreme_value=peak_data.last[index_all]
    extreme_vol.loc[object_stock[:-7],'local_peak_vol']=np.sqrt((np.diff(np.log(list(extreme_value)))**2).sum())


The following dataframe shows the calculated number of four metrics of different stocks on different date.

In [10]:
print(extreme_vol)

                   Parkinson Garman_Klass Rogers_Satchell local_peak_vol
TRV_2019-05-28   0.000283762  0.000236233     0.000376184      0.0128039
MMM_2019-05-28   0.000356989  0.000331983     0.000458151      0.0128056
MMM_2019-05-24   0.000473722  0.000447043       0.0006702      0.0136502
MSFT_2019-05-29  0.000541447  0.000520891     0.000748425      0.0174715
TRV_2019-05-24   0.000278438  0.000238676     0.000367571       0.010566
MCD_2019-05-23   0.000307933  0.000267885     0.000413383      0.0124092
VZ_2019-05-29    0.000470771  0.000430031     0.000649193      0.0165103
AXP_2019-05-23   0.000382019  0.000340742     0.000522184      0.0146665
INTC_2019-05-28  0.000715759  0.000681806     0.000945222       0.033675
INTC_2019-05-24  0.000559615  0.000528001      0.00076548      0.0173916
CAT_2019-05-23   0.000657662  0.000574821     0.000913742       0.023919
PG_2019-05-29    0.000406488  0.000349323     0.000558017      0.0158274
WMT_2019-05-24   0.000328471  0.000300579     0.000

In [12]:
extreme_vol.astype("float").corr()

,Parkinson,Garman_Klass,Rogers_Satchell,local_peak_vol
Parkinson,1.000000,0.985847,0.992554,0.872398
Garman_Klass,0.985847,1.000000,0.985662,0.837739
Rogers_Satchell,0.992554,0.985662,1.000000,0.844956
local_peak_vol,0.872398,0.837739,0.844956,1.000000


From the correlation of four metrics, we can also find they are highly correlated, which proves that they can catch similarities of intraday volatilities.

Reference: https://eranraviv.com/intraday-volatility-measures/